In [1]:
import os
import pandas as pd
import pickle
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600

In [2]:
explainer_map = {'conceptx': 'ConceptX', 'aconceptx': 'AntonymConceptX', 'conceptx_r': 'ConceptX-R', 'conceptx_a': 'ConceptX-A', 'conceptshap': 'ConceptSHAP', 'tokenshap': 'TokenSHAP', 'random': 'Random'}
explainer_order = ["Random", "TokenSHAP", "ConceptSHAP", "ConceptX", 'AntonymConceptX', "ConceptX-R", "ConceptX-A"]
MODEL_NAMES = {"gpt2": "GPT-2", "gemma-2-2b":"Gemma-2-2B", "gemma-3-4b":"Gemma-3-4B", "gpt4o-mini": "GPT-4o-mini", "llama-3-3b": "Llama-3.2-3B"}

In [3]:
save_dir = "/cluster/home/kamara/conceptx"
seed_ = 0

In [11]:
folder_path = os.path.join(save_dir, f"results-no-sample/classification")  # Replace with your folder path

# Initialize a list to store the dataframes
df_list = []

# Loop through each file in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        full_path = os.path.join(root, file)
        parts = file.split('_')
        print("parts: ", parts)
        
        df = pd.read_csv(full_path)

        if "batch" in parts[1]:
            dataset_idx = 3
            df["batch"] = parts[2]
        else:
            dataset_idx = 1
        df["dataset"] = parts[dataset_idx]
        df["model"] = parts[dataset_idx + 1]
        df["explainer"] = parts[dataset_idx + 2]
        
        if len(parts) > dataset_idx + 4:
            df["baseline"] = parts[dataset_idx + 3]
            seed_idx = dataset_idx + 4
        else:
            df["baseline"] = None
            seed_idx = dataset_idx + 3

        df["seed"] = parts[seed_idx].split(".")[0]
        df_list.append(df)

# Concatenate all dataframes in the list
df_all = pd.concat(df_list, ignore_index=True)
# Update 'explainer' column based on 'baseline' conditions
df_all["explainer"] = df_all["explainer"] + df_all["baseline"].map({"reference": "_r", "aspect": "_a"}).fillna("")
df_all

parts:  ['classification', 'batch', '0', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['classification', 'batch', '1', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['classification', 'batch', '2', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['classification', 'batch', '3', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['classification', 'batch', '4', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['classification', 'batch', '5', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['classification', 'batch', '9', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['classification', 'batch', '7', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['classification', 'batch', '6', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['classification', 'batch', '8', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['classification', 'batch', '1', 'sst2', 'mistral-7b-it', 'random', '2.csv']
parts:  ['classification', 'batch', '0', 'sst2', 'mistral-7b-it',

,id,input,p0,p_highest,p_label,aspect,highest_token,label,batch,dataset,model,explainer,baseline,seed
0,5207,into a future they wo n't much care about,0.573470,0.684823,0.363555,negative,into,care,0,sst2,mistral-7b-it,random,None,0
1,3722,"its true-to-life characters ,",0.312101,0.417399,0.088916,positive,its,true-to-life,0,sst2,mistral-7b-it,random,None,0
2,2440,while the performances are often engaging,0.875884,0.396348,0.096676,positive,performances,engaging,0,sst2,mistral-7b-it,random,None,0
3,1147,we 've seen the hippie-turned-yuppie plot befo...,0.048633,0.080064,0.048633,negative,before,negative,0,sst2,mistral-7b-it,random,None,0
4,4116,"first-rate , especially sorvino",0.033138,0.114857,0.220967,positive,first,first-rate,0,sst2,mistral-7b-it,random,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17215,4549,cut their losses -- and ours -- and,0.204740,0.173384,0.334392,negative,losses,cut,9,sst2,mistral-7b-it,tokenshap,None,1
17216,3283,is matched only by the ridiculousness of its p...,0.818631,0.080190,0.080190,negative,ridiculousness,ridiculousness,9,sst2,mistral-7b-it,tokenshap,None,1
17217,308,demands and receives excellent performances,0.919406,0.082623,0.082623,positive,excellent,excellent,9,sst2,mistral-7b-it,tokenshap,None,1
17218,1352,draws on an elegant visual sense and a talent,0.719259,0.406884,0.269476,positive,talent,elegant,9,sst2,mistral-7b-it,tokenshap,None,1


In [12]:
### Size of dataset for model
model_list = ["llama-3-3b", "gemma-3-4b", "gpt4o-mini", "mistral-7b-it"]
df = df_all[df_all["model"].isin(model_list)]
df_stats = df.groupby(["dataset", "model", "seed", "explainer"])["id"].nunique().reset_index()

In [13]:
dataset = "sst2"
df_stats[df_stats['dataset']==dataset]

,dataset,model,seed,explainer,id
0,sst2,mistral-7b-it,0,aconceptx,943
1,sst2,mistral-7b-it,0,conceptshap,963
2,sst2,mistral-7b-it,0,conceptx,925
3,sst2,mistral-7b-it,0,conceptx-a,926
4,sst2,mistral-7b-it,0,random,1000
5,sst2,mistral-7b-it,0,tokenshap,1000
6,sst2,mistral-7b-it,1,aconceptx,932
7,sst2,mistral-7b-it,1,conceptshap,954
8,sst2,mistral-7b-it,1,conceptx,918
9,sst2,mistral-7b-it,1,conceptx-a,918


In [14]:
# Convert relevant columns to numeric
df_all["p0"] = pd.to_numeric(df_all["p0"], errors="coerce")
df_all["p_highest"] = pd.to_numeric(df_all["p_highest"], errors="coerce")
df_all["p_label"] = pd.to_numeric(df_all["p_label"], errors="coerce")


In [15]:
# Filter only by dataset and p0 threshold (no longer filtering by model)
df = df_all[(df_all["dataset"] == dataset)]
df = df[df["p0"] > 0.5]

# Compute diffs
df["p0_p_highest_diff"] = df["p0"] - df["p_highest"]
df["p0_p_label_diff"] = df["p0"] - df["p_label"]

# Compute stats: group by explainer and model
stats = df.groupby(["explainer", "model", "seed"])[["p0_p_highest_diff"]].mean().reset_index()#.agg(["mean", "var"])
stats = stats.groupby(["explainer", "model"])[["p0_p_highest_diff"]].agg(["mean", "var"])
stats

p0_p_highest_diff          
                                       mean       var
explainer   model                                    
aconceptx   mistral-7b-it          0.231727  0.000126
conceptshap mistral-7b-it          0.246585  0.000021
conceptx    mistral-7b-it          0.253148  0.000197
conceptx-a  mistral-7b-it          0.227053  0.000088
random      mistral-7b-it          0.133003  0.000166
tokenshap   mistral-7b-it          0.235620  0.000110

In [16]:
# Compute stats: group by explainer and model
df_label = df[['model', 'explainer', 'seed', "p0_p_label_diff"]]
df_label = df_label.rename(columns={'p0_p_label_diff': 'p0_p_highest_diff'})
stats_label = df_label.groupby(["model", "seed"])[["p0_p_highest_diff"]].mean()
stats_label = stats_label.groupby(["model"])[["p0_p_highest_diff"]].agg(["mean", "var"])
# Step 2: Add new level to index to create the new explainer row
stats_label["explainer"] = "gpt4o-mini_explainer"
stats_label = stats_label.set_index("explainer", append=True)
stats_label.index = stats_label.index.reorder_levels(["explainer", "model"])
stats_label

p0_p_highest_diff          
                                                mean       var
explainer            model                                    
gpt4o-mini_explainer mistral-7b-it          0.416866  0.000015

In [23]:


# Step 3: Concatenate with df1
df_combined = pd.concat([stats, stats_label])

# Step 4: Optional: sort for clarity
df_combined = df_combined.sort_index()


# Flatten multi-index columns
df_combined.columns = [f"{model}_{stat}" for metric, stat in df_combined.columns for model in [metric.split("_diff")[0]]]

# Pivot so each model becomes a column group
df_combined = df_combined.unstack("model")

# Reorder and rename explainers
desired_order = ["random", "tokenshap", "conceptshap", "conceptx", "aconceptx", "conceptx_a", "gpt4o-mini_explainer"]
explainer_map = {
    'conceptx': 'ConceptX',
    'aconceptx': 'AntonymConceptX',
    'conceptx_r': 'ConceptX-R',
    'conceptx_a': 'ConceptX-A',
    'conceptshap': 'ConceptSHAP',
    'tokenshap': 'TokenSHAP',
    'random': 'Random',
    'gpt4o-mini_explainer': 'GPT-4o mini'
}
df_combined = df_combined.rename(index=explainer_map)
df_combined = df_combined.reindex([explainer_map.get(e, e) for e in desired_order])

df_combined.columns = df_combined.columns.swaplevel()
df_combined = df_combined.sort_index(axis=1)
df_combined

model                  gemma-3-4b                         gpt4o-mini  \
                p0_p_highest_mean p0_p_highest_var p0_p_highest_mean   
explainer                                                              
Random                   0.136672         0.068833          0.137902   
TokenSHAP                0.384658         0.139008          0.428577   
ConceptSHAP              0.321727         0.136756          0.367423   
ConceptX                 0.294545         0.130210          0.308379   
AntonymConceptX          0.386466         0.145040          0.346844   
ConceptX-A               0.236690         0.112373          0.300422   
GPT-4o mini              0.502124         0.127368          0.500553   

model                                   llama-3-3b                   
                p0_p_highest_var p0_p_highest_mean p0_p_highest_var  
explainer                                                            
Random                  0.070212          0.135650         0.066046  
TokenSHAP               0.135817          0.155412         0.071190  
ConceptSHAP             0.139335          0.175997         0.091633  
ConceptX                0.134380          0.203254         0.098645  
AntonymConceptX         0.135727          0.221914         0.105111  
ConceptX-A              0.129533          0.193819         0.096515  
GPT-4o mini             0.127427          0.500327         0.129132

In [24]:

MODEL_NAMES = {"gpt2": "GPT-2", "gemma-2-2b":"Gemma-2-2B", "gemma-3-4b":"Gemma-3-4B", "gpt4o-mini": "GPT-4o mini", "llama-3-3b": "Llama-3.2-3B"}
model_map = MODEL_NAMES
model_order = ["Llama-3.2-3B", "Gemma-3-4B", "GPT-4o mini"]


# Rename models in column MultiIndex
df_combined.columns = pd.MultiIndex.from_tuples([
    (model_map.get(model, model), metric) for model, metric in df_combined.columns
])

# Reorder columns according to model_order
df_combined = df_combined[model_order]
df_combined

Llama-3.2-3B                         Gemma-3-4B  \
                p0_p_highest_mean p0_p_highest_var p0_p_highest_mean   
explainer                                                              
Random                   0.135650         0.066046          0.136672   
TokenSHAP                0.155412         0.071190          0.384658   
ConceptSHAP              0.175997         0.091633          0.321727   
ConceptX                 0.203254         0.098645          0.294545   
AntonymConceptX          0.221914         0.105111          0.386466   
ConceptX-A               0.193819         0.096515          0.236690   
GPT-4o mini              0.500327         0.129132          0.502124   

                                       GPT-4o mini                   
                p0_p_highest_var p0_p_highest_mean p0_p_highest_var  
explainer                                                            
Random                  0.068833          0.137902         0.070212  
TokenSHAP               0.139008          0.428577         0.135817  
ConceptSHAP             0.136756          0.367423         0.139335  
ConceptX                0.130210          0.308379         0.134380  
AntonymConceptX         0.145040          0.346844         0.135727  
ConceptX-A              0.112373          0.300422         0.129533  
GPT-4o mini             0.127368          0.500553         0.127427